# MAXELLA TFRS: Multi-Task Model - Joint Model

## The Two-Tower and Ranking Models


## Contents 


 * Introduction
 * Dataset
 * Sourcing and Loading
 * A Multi-Task Model
   * Rating-specialized Model
   * Retrieval-specialized Model
   * Joint Model
 * Modeling Summary
 * Next Step - Joint Model Tuning
   * Adding more features
   * Optimize Embedding
   * embedding_dimension 32 to 64
   * epochs= 3 to 32
   * Learning rate
  
    

## 1. Introduction 

In the [TFRS Modeling - Two-Tower](https://github.com/akthammomani/MAXELLA-APP-Movies-Tensorflow-Recommenders-TFRS/tree/main/Notebooks/TFRS-Modelling) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In addition, building a joint model for multiple tasks may produce better results than building a number of task-specific models. This is especially true where some data is abundant (for example, clicks), and some data is sparse (purchases, returns, manual reviews). In those scenarios, a joint model may be able to use representations learned from the abundant task to improve its predictions on the sparse task via a phenomenon known as [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). For example, [this paper](https://openreview.net/pdf?id=SJxPVcSonN) shows that a model predicting explicit user ratings from sparse user surveys can be substantially improved by adding an auxiliary task that uses abundant click log data.

In this Notebook, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## 2. Dataset: 

**Movie Lens** contains a set of movie ratings from the MovieLens website, a movie recommendation service. This dataset was collected and maintained by [GroupLens](https://grouplens.org/) , a research group at the University of Minnesota. There are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all datasets, the movies data and ratings data are joined on "movieId". The 25m dataset, latest-small dataset, and 20m dataset contain only movie data and rating data. The 1m dataset and 100k dataset contain demographic data in addition to movie and rating data.

**movie_lens/1m** can be treated in two ways:

  * It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of *implicit feedback*, where users' watches tell us which things they prefer to see and which they'd rather not see (This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example).
  * It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of *explicit feedback*: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.



**(a) [movie_lens/1m-ratings](https://www.tensorflow.org/datasets/catalog/movie_lens#movie_lens1m-ratings):**
 * Config description: This dataset contains 1,000,085 anonymous ratings of approximately 3,619 movies made by 6,040 MovieLens users who joined MovieLens. Ratings are in whole-star increments. This dataset contains demographic data of users in addition to data on movies and ratings.
 * This dataset is the largest dataset that includes demographic data from movie_lens.
 * "user_gender": gender of the user who made the rating; a true value corresponds to male
 * "bucketized_user_age": bucketized age values of the user who made the rating, the values and the corresponding ranges are:
   * 1: "Under 18"
   * 18: "18-24"
   * 25: "25-34"
   * 35: "35-44"
   * 45: "45-49"
   * 50: "50-55"
   * 56: "56+"
 * "user_occupation_label": the occupation of the user who made the rating represented by an integer-encoded label; labels are preprocessed to be consistent across different versions
 * "user_occupation_text": the occupation of the user who made the rating in the original string; different versions can have different set of raw text labels
 * "user_zip_code": the zip code of the user who made the rating.
 * "release_date": This is the movie release date, in unix epoch (UTC - units of seconds) (int64).
 * "director": This is the director of the movie.
 * "start": This is the main star of the movie.
 
 **(b) [movie_lens/1m-movies](https://www.tensorflow.org/datasets/catalog/movie_lens#movie_lens1m-movies):**

 * Config description: This dataset contains data of approximately 3,619 movies rated in the 1m dataset.
 * Download size: 5.64 MiB
 * Dataset size: 351.12 KiB
 * Auto-cached ([documentation](https://www.tensorflow.org/datasets/performances#auto-caching)): Yes
 * Features:
```
FeaturesDict({
              'movie_genres': Sequence(ClassLabel(shape=(), dtype=tf.int64, num_classes=21)),
              'movie_id': tf.string,
              'movie_title': tf.string,
             })
```


## 3. Sourcing and Loading

### 3.1 Import relevant libraries

In [1]:
# Import the necessary Libararies: 

import os
import pprint
import tempfile
import matplotlib.pyplot as plt
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from typing import Dict, Text
import pandas as pd
import numpy as np

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

plt.style.use('ggplot')

### 3.2 Preparing the dataset

Let's first have a look at the data.

We use the MovieLens dataset from Tensorflow Datasets. Loading **[movie_lens/1m-ratings](https://www.tensorflow.org/datasets/catalog/movie_lens#movie_lens1m-ratings)** yields a ***tf.data.Dataset*** object containing the ratings data and loading **[movie_lens/1m-movies](https://www.tensorflow.org/datasets/catalog/movie_lens#movie_lens1m-movies)** yields a ***tf.data.Dataset*** object containing only the movies data.

Note that since the MovieLens dataset does not have predefined splits, all data are under train split.

In [2]:
# Now, let's look at the final dataframe where we merged tensorflow dataset with movies_metadata.csv and credits.csv**
# let's make sure to use encoding argument to avoid in encoding errors !!!
ratings = pd.read_csv('ratings.csv', encoding='ISO-8859-1')
movies = pd.read_csv('movies.csv', encoding='ISO-8859-1')

In [3]:
# now let's look at the new features we added: 'movie_imdb_id', 'cast', 'director', 'cast_size', 'crew_size', 'imdb_id' and 'release_date':
#Also, please note that all wrong spelled movies are corrected!!!!
ratings.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,director,release_date,star
0,50,[7],1251,Eight and half,974089380,False,2497,14,sales/marketing,3,37922,Federico Fellini,-217123200,Marcello Mastroianni
1,18,[7],1251,Eight and half,986722200,True,671,17,college/grad student,5,61761,Federico Fellini,-217123200,Marcello Mastroianni
2,45,[7],1251,Eight and half,960071880,False,5590,12,programmer,2,94117,Federico Fellini,-217123200,Marcello Mastroianni
3,25,[7],1251,Eight and half,1011993120,True,1851,20,unemployed,5,59602,Federico Fellini,-217123200,Marcello Mastroianni
4,35,[7],1251,Eight and half,963100320,False,5526,1,artist,5,27514,Federico Fellini,-217123200,Marcello Mastroianni


In [4]:
# now let's look at the new features we added: 'movie_imdb_id', 'cast', 'director', 'cast_size', 'crew_size', 'imdb_id' and 'release_date':
#Also, please note that all wrong spelled movies are corrected!!!!
movies.head()

,movie_genres,movie_id,movie_title
0,[7],1251,Eight and half
1,[7],2188,54
2,[7],1609,One Eight Seven
3,"[13, 15]",2311,2010
4,"[3, 4]",2031,The Million Dollar Duck


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000085 entries, 0 to 1000084
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   bucketized_user_age    1000085 non-null  int64 
 1   movie_genres           1000085 non-null  object
 2   movie_id               1000085 non-null  int64 
 3   movie_title            1000085 non-null  object
 4   timestamp              1000085 non-null  int64 
 5   user_gender            1000085 non-null  bool  
 6   user_id                1000085 non-null  int64 
 7   user_occupation_label  1000085 non-null  int64 
 8   user_occupation_text   1000085 non-null  object
 9   user_rating            1000085 non-null  int64 
 10  user_zip_code          1000085 non-null  int64 
 11  director               1000085 non-null  object
 12  release_date           1000085 non-null  int64 
 13  star                   1000085 non-null  object
dtypes: bool(1), int64(8), object(5)
me

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3658 entries, 0 to 3657
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_genres  3658 non-null   object
 1   movie_id      3658 non-null   int64 
 2   movie_title   3658 non-null   object
dtypes: int64(1), object(2)
memory usage: 85.9+ KB


In [7]:
# movie_id from int to str:
ratings['movie_id'] = ratings['movie_id'].astype('str')

In [8]:
# movie_id from int to str:
movies['movie_id'] = movies['movie_id'].astype('str')

In [9]:
# user_id from int to str:
ratings['user_id'] = ratings['user_id'].astype('str')

In [10]:
# user_zip_code from int to str:
ratings['user_zip_code'] = ratings['user_zip_code'].astype('str')

In [11]:
#let's have a general view of main_df:
ratings_missing = pd.concat([ratings.nunique(), ratings.dtypes, ratings.isnull().sum(), 100*ratings.isnull().mean()], axis=1)
ratings_missing.columns = [['count', 'data_type', 'missing_count', 'missing%']]
ratings_missing

,count,data_type,missing_count,missing%
bucketized_user_age,7,int64,0,0.0
movie_genres,302,object,0,0.0
movie_id,3656,object,0,0.0
movie_title,3619,object,0,0.0
timestamp,174246,int64,0,0.0
user_gender,2,bool,0,0.0
user_id,6040,object,0,0.0
user_occupation_label,21,int64,0,0.0
user_occupation_text,21,object,0,0.0
user_rating,5,int64,0,0.0


In [14]:
#let's have a general view of main_df:
movies_missing = pd.concat([movies.nunique(), movies.dtypes, movies.isnull().sum(), 100*movies.isnull().mean()], axis=1)
movies_missing.columns = [['count', 'data_type', 'missing_count', 'missing%']]
movies_missing

,count,data_type,missing_count,missing%
movie_genres,302,object,0,0.0
movie_id,3656,object,0,0.0
movie_title,3619,object,0,0.0


In [15]:
#let's wrap the **pandas dataframe** into **tf.data.Dataset** object using **tf.data.Dataset.from_tensor_slices** using: tf.data.Dataset.from_tensor_slices
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

In [16]:
#The ratings dataset returns a dictionary of movie id, user id, the assigned rating, timestamp, movie information, and user information:
#View the data from ratings dataset:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 50,
 'director': b'Federico Fellini',
 'movie_genres': b'[7]',
 'movie_id': b'1251',
 'movie_title': b'Eight and half ',
 'release_date': -217123200,
 'star': b'Marcello Mastroianni',
 'timestamp': 974089380,
 'user_gender': False,
 'user_id': b'2497',
 'user_occupation_label': 14,
 'user_occupation_text': b'sales/marketing',
 'user_rating': 3,
 'user_zip_code': b'37922'}


In [17]:
#The ratings dataset returns a dictionary of movie id, user id, the assigned rating, timestamp, movie information, and user information:
#View the data from ratings dataset:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': b'[7]', 'movie_id': b'1251', 'movie_title': b'Eight and half '}


In [18]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})


movies = movies.map(lambda x: x["movie_title"])

In [19]:
# let's use a random split, putting 75% of the ratings in the train set, and 25% in the test set:
# Assign a seed=42 for consistency of results and reproducibility:
seed = 42
l = len(ratings)

tf.random.set_seed(seed)
shuffled = ratings.shuffle(l, seed=seed, reshuffle_each_iteration=False)

#Save 75% of the data for training and 25% for testing:
train_ = int(0.75 * l)
test_ = int(0.25 * l)

train = shuffled.take(train_)
test = shuffled.skip(train_).take(test_)

In [20]:
# Now, let's find out how many uniques users/movies:
movie_titles = movies.batch(l)
user_ids = ratings.batch(l).map(lambda x: x["user_id"])

#Movies uniques:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))

#users unique
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# take a look at the movies:
unique_movie_titles[:10]

array([b' Pret-a-Porter (Ready to Wear)', b"'night, Mother",
       b'...And God Created Woman (Et Dieu... crea la femme)',
       b'...And Justice for All', b'1-900', b'10 Things I Hate About You',
       b'101 Dalmatians', b'12 Angry Men', b'2 Days in the Valley',
       b'2 or 3 Things I Know About Her'], dtype=object)

## 4. A Multi-Task Model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

Now, let's define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

In [23]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add 1 to account for the unknown token.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

However, now we will have two tasks. The first is the rating task:

In [24]:
tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

In [25]:
tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128)
    )
)

As before, this task's goal is to predict which movies the user will or will not watch.

**Putting it together**

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [28]:
class MovielensModel(tfrs.models.Model):
    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
        # We take the loss weights in the constructor: this allows us to instantiate
        # several model objects with different loss weights.

        super().__init__()

        embedding_dimension = 32

        # User and movie models.
        self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_movie_titles, mask_token=None),
          tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        ])
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # A small model to take in user and movie embeddings and predict ratings.
        # We can make this as complicated as we want as long as we output a scalar
        # as our prediction.
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])

        # The tasks.
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.movie_model)
            )
        )

        # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])
        # And pick out the movie features and pass them into the movie model.
        movie_embeddings = self.movie_model(features["movie_title"])

        return (
            user_embeddings,
            movie_embeddings,
            # We apply the multi-layered rating model to a concatentation of
            # user and movie embeddings.
            self.rating_model(
                tf.concat([user_embeddings, movie_embeddings], axis=1)
            ),
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        ratings = features.pop("user_rating")

        user_embeddings, movie_embeddings, rating_predictions = self(features)

        # We compute the loss for each task.
        rating_loss = self.rating_task(
                                      labels=ratings,
                                      predictions=rating_predictions,
                                      )
        retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

        # And combine them using the loss weights.
        return (self.rating_weight * rating_loss
                + self.retrieval_weight * retrieval_loss)

### 4.1 Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [29]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [30]:
# Then shuffle, batch, and cache the training and evaluation data:
# Segment the batches so that the model runs 13 training batches (2^13) and 11 test batches (2^11) per epoch, 
# while having a batch size which is a multiple of 2^n.
cached_train = train.shuffle(l).batch(8192).cache()
cached_test = test.batch(2048).cache()

In [31]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
92/92 [==============================] - 257s 3s/step - root_mean_squared_error: 1.2577 - factorized_top_k/top_1_categorical_accuracy: 1.8798e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0029 - factorized_top_k/top_50_categorical_accuracy: 0.0145 - factorized_top_k/top_100_categorical_accuracy: 0.0284 - loss: 1.5751 - regularization_loss: 0.0000e+00 - total_loss: 1.5751
Epoch 2/3
92/92 [==============================] - 218s 2s/step - root_mean_squared_error: 1.0390 - factorized_top_k/top_1_categorical_accuracy: 2.7864e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0039 - factorized_top_k/top_50_categorical_accuracy: 0.0174 - factorized_top_k/top_100_categorical_accuracy: 0.0328 - loss: 1.0783 - regularization_loss: 0.

Alright, here we get good RMSE but with poor prediction

### 4.2 Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [32]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [33]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
92/92 [==============================] - 291s 3s/step - root_mean_squared_error: 3.8135 - factorized_top_k/top_1_categorical_accuracy: 7.5994e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0072 - factorized_top_k/top_10_categorical_accuracy: 0.0155 - factorized_top_k/top_50_categorical_accuracy: 0.0745 - factorized_top_k/top_100_categorical_accuracy: 0.1346 - loss: 71210.2944 - regularization_loss: 0.0000e+00 - total_loss: 71210.2944
Epoch 2/3
92/92 [==============================] - 301s 3s/step - root_mean_squared_error: 3.8523 - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0126 - factorized_top_k/top_10_categorical_accuracy: 0.0257 - factorized_top_k/top_50_categorical_accuracy: 0.1083 - factorized_top_k/top_100_categorical_accuracy: 0.1860 - loss: 68970.8301 - regularization_loss: 0.0000e+00 - total_loss: 68970.8301
Epoch 3/3
123/123 [==============================] - 101s 802ms/step - root_mean_squared_error: 

Interesting, now we're getting the opposite results, poor RMSE but with better predictions

### 4.3  Joint model

Let's now train a model that assigns positive weights to both tasks.

In [34]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [35]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
92/92 [==============================] - 302s 3s/step - root_mean_squared_error: 1.2950 - factorized_top_k/top_1_categorical_accuracy: 6.7328e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0067 - factorized_top_k/top_10_categorical_accuracy: 0.0147 - factorized_top_k/top_50_categorical_accuracy: 0.0712 - factorized_top_k/top_100_categorical_accuracy: 0.1286 - loss: 71276.6060 - regularization_loss: 0.0000e+00 - total_loss: 71276.6060
Epoch 2/3
92/92 [==============================] - 293s 3s/step - root_mean_squared_error: 1.0012 - factorized_top_k/top_1_categorical_accuracy: 0.0013 - factorized_top_k/top_5_categorical_accuracy: 0.0125 - factorized_top_k/top_10_categorical_accuracy: 0.0253 - factorized_top_k/top_50_categorical_accuracy: 0.1076 - factorized_top_k/top_100_categorical_accuracy: 0.1847 - loss: 69020.9598 - regularization_loss: 0.0000e+00 - total_loss: 69020.9598
Epoch 3/3
123/123 [==============================] - 91s 718ms/step - root_mean_squared_error: 0

Better, over all results compared to the previous models.

## 5. Modeling Summary

Alright, as shown below joint model managed to provide both better prediction with low RMSE, as shown below:

 |Models|Retrieval top-100 accuracy | Ranking RMSE|
 |:--:|:--:|:--:|
  |Model 1: Rating-specialized model|0.031|0.940|
 |Model 2: Retrieval-specialized model|0.158|3.868|
 |Model 3: Joint model|0.158|0.974|


## 6. Next Step - Joint Model Tuning

Alright, let's focus in tuning the joint model due to the high potential, so, we'll do the following: 

 * Adding more features
 * Optimize Embedding
 * embedding_dimension 32 to 64
 * epochs= 3 to 32
 * Learning rate

In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))